In [ ]:
# libraries
import numpy as np
import pandas as pd
import networkx as nx
import pickle as pk
from glob import glob
import os, json, random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime,date, timedelta
from matplotlib import pyplot as plt
import matplotlib
from datetime import date
import re,ast,random
import matplotlib.dates as mdates
matplotlib.rcParams['font.size'] = 12
matplotlib.rcParams['font.family'] = 'Arial'
import seaborn as sns
from scipy.stats import mannwhitneyu
from collections import Counter 
import matplotlib
import calendar
import matplotlib.font_manager as font_manager

In [ ]:
# extract data:

new_sample =False
if new_sample:
    cleaned_data = []
    for i in range(3):
        init_data = pd.read_csv('cleaned_data_part-'+str(i)+'.csv',lineterminator='\n')
        init_data['date'] = [date.fromtimestamp(t) if str(t) !='nan' else np.nan for t in init_data['time_epoch'].values]
        init_data['tweetId'] = init_data['tweetId'].astype(str)
        init_data['conversation_id'] = init_data['conversation_id'].astype(str)
        init_data['twitterAuthorScreenname'] = init_data['twitterAuthorScreenname'].astype(str)
        init_data = init_data.drop_duplicates(subset='tweetId')
        cleaned_data.append(init_data)
    cleaned_data = pd.concat(cleaned_data)
    del init_data
    cleaned_data = cleaned_data.drop_duplicates(subset='tweetId')
    init_data0 = cleaned_data.loc[cleaned_data['date']<=datetime(2022,12,1).date(),].sample(frac=0.2)
    init_data1 = cleaned_data.loc[cleaned_data['date']>datetime(2022,12,1).date(),]    
    del cleaned_data
    cleaned_data = pd.concat([init_data0,init_data1])
    user_cleaned_data = cleaned_data.groupby('twitterAuthorScreenname')
    
    
    cleaned_data.to_csv('equal_sample.csv',index=False)
else:
    cleaned_data = pd.read_csv('equal_sample.csv',lineterminator='\n')#'sampled_data_3.csv',lineterminator='\n')
    cleaned_data['twitterAuthorScreenname'] = cleaned_data['twitterAuthorScreenname'].astype(str).values
    cleaned_data['date'] = [date.fromtimestamp(t) if str(t) !='nan' else np.nan for t in cleaned_data['time_epoch'].values]
    user_cleaned_data = cleaned_data.groupby('twitterAuthorScreenname')
    if False:
        coord_data_hashtag = []
        coord_data_rt = []
        coord_data_time = []    
        for i in range(3):
            part_data = pd.read_csv('cleaned_data_part-'+str(i)+'.csv',lineterminator='\n')
            part_data['twitterAuthorScreenname'] = part_data['twitterAuthorScreenname'].astype(str)
            part_users = set(part_data['twitterAuthorScreenname'].drop_duplicates().values.tolist())
            user_part_data = part_data.groupby('twitterAuthorScreenname')
            coord_data_hashtag.append(pd.concat([user_part_data.get_group(u) for u in hashtag_G.nodes() if u in part_users]))
            coord_data_rt.append(pd.concat([user_part_data.get_group(u) for u in retweet_G.nodes() if u in part_users]))
            coord_data_time.append(pd.concat([user_part_data.get_group(u) for u in time_G.nodes() if u in part_users]))
        coord_data_hashtag = pd.concat(coord_data_hashtag)
        coord_data_rt = pd.concat(coord_data_rt)
        coord_data_time = pd.concat(coord_data_time)
print(len(cleaned_data))
cleaned_data = cleaned_data.drop_duplicates(subset='tweetId')
print(len(cleaned_data))


In [ ]:
# find random users:
# see: InauthenticAccountReplication.ipynb for extracting hashtag_G, retweet_G, and time_G
# we leave it out here for brevity
for G2 in [hashtag_G, retweet_G,time_G]:
    nodes=list(G2.nodes())
    random.shuffle(nodes)
    print('{')
    for n in nodes[:100]:
        print(str(n)+': ,')
    print('}\n=======\n\n\n======')
sample_users = cleaned_data['twitterAuthorScreenname'].drop_duplicates().sample(100).values.tolist()
print('random={')
for n in sample_users:
    print(str(n)+': ,')
print('}==========\n\n\n\n======')

sample_users = cleaned_data.loc[(cleaned_data['date']<datetime(2022,10,27).date()) & (cleaned_data['date']>datetime(2022,4,15).date()),'twitterAuthorScreenname'].drop_duplicates().sample(100).values.tolist()
print('random_before={')
for n in sample_users:
    print(str(n)+': ,')
print('}==========\n\n\n\n======')

sample_users = cleaned_data.loc[(cleaned_data['date']>datetime(2022,10,27).date()) & ((cleaned_data['date']<datetime(2022,12,1).date()) | (cleaned_data['date']>datetime(2022,12,31).date())),'twitterAuthorScreenname'].drop_duplicates().sample(100).values.tolist()
print('random_after={')
for n in sample_users:
    print(str(n)+': ,')
print('}==========\n\n\n\n======')

sample_users = cleaned_data.dropna(subset='cashtags')['twitterAuthorScreenname'].drop_duplicates().sample(100).values
print('random_cashtag={')
for n in sample_users:
    print(str(n)+': ,')
print('}==========\n\n\n\n======')


sample_users = cleaned_data.loc[(cleaned_data['date']<datetime(2022,10,27).date()) & (cleaned_data['date']>datetime(2022,4,15).date()),'twitterAuthorScreenname'].sample(100).values.tolist()
print('random_weighted_before={')
for n in sample_users:
    print(str(n)+': ,')
print('}==========\n\n\n\n======')

sample_users = cleaned_data.loc[(cleaned_data['date']>datetime(2022,10,27).date()) & ((cleaned_data['date']<datetime(2022,12,1).date()) | (cleaned_data['date']>datetime(2022,12,31).date())),'twitterAuthorScreenname'].sample(100).values.tolist()
print('random_weighted_after={')
for n in sample_users:
    print(str(n)+': ,')
print('}==========\n\n\n\n======')

In [ ]:
# upload annotated data (bot scores for each user)

# bot score of accounts weighted by their activity. 
# bot score of random posts instead of random accounts
random_weighted_before={
735963557301489665: 1.5,
156962894: 0.2,
1030889535813636096: 1,
1286783146378305538: 1.4,
1338550958234738688: 1.1,
245037893: 0.4,
1152990609088454656: 1.1,
987339281025961989: 1.3,
1528344912244183049: 1.3,
761046956655284229: 0.2,
904090399: 0.1,
544874646: 0.7,
163150799: 1.1,
1279697699286155264: 0.8,
1026992209546108929: 0.1,
14093253: 0.4,
808746737275994112: 0.1,
3166064574: 0.1,
1346676189998559232: 1.9,
884043121180499968: 0.4,
26945111: 0.1,
118839419: 0.3,
937727756934307841: 0.7,
1430652951715057667: 0.6,
1554589779203080192: 1.6,
1000435661231607813:2.2,
723537344754798592: 1.9,
100678690: 1.2,
1473466366288019461: 1.6,
771255477749420032: 0.6,
1241290412: 1.3,
841213176: 0.1,
765553492589178881: 0.9,
1235291641667301378: 0.3,
865670126296719360: 0.8,
581482097: 0.1,
1197730028164808704: 1.9,
838203047104765952: 1.8,
155954704: 0.4,
2345445539: 0,
1489933937006710784: 1.7,
1326662243103236099: 1.4,
2701718326: 0.5,
289092705: 0.9,
1432074242246381574: 1.3,
814751374227415040: 4.1,
1146383263876952065: 2,
1548340421603893248: 1.1,
1478633551000186880: 2.3,
3952198996: 0.4,
4563095379: 0.9,
3900339803: 1.4,
1105278899368321024: 1.3,
44917376: 0.4,
969472332522803200: 0.1,
1328422666198544385: 2.5,
1354717353959940096: 1.9,
1654582129: 0.3,
1324996718128148482: 0.3,
1161487577121271811: 0.9,
2275168742: 0.3,
1266682488719650816: 0.8,
177413064: 0.6,
1106828841316765696: 0.1,
1616558768: 1.1,
1509490747506294785: 1.3,
1358947116006080512: 1.5,
919908152: 0.3,
14421107: 0.1,
2200106636: 0.5,
4618648394: 0.1,
2811579568: 1.3,
971868672523456513: 1.4,
1902899629: 0,
1384610111948218371: 0.1,
14847904: 0.4,
2322721982: 0.1,
1527298907423887360: 1.8,
1176797971876237312: 1.8,
1033463191: 0.2,
1218899828077400064: 0.3,
1504988542400970757: 0.2,
2172771908: 2.3,
1380752066: 0.1,
1433160278934376451: 1.1,
896576846686539776: 0.4,
1048587643762688000: 0.7,
15318109: 0.1,
1150321171784765440: 1.6,
2583324551: 1.9,
1174114112302792704: 3.7,
3995589255: 2.8,
773694348672335872: 1,
50491680: 0.6,
2837520220: 1.9,
1096147944: 1.1,
951522626: 0.8,
1165686386592288770: 2.5,
21287922: 0.5,
1181636450191904769: 1.8,
}

random_weighted_after = {
1589473790269870080: 1.6,
1211974881077710851: 0.4,
2815096616: 0.4,
723844006896123906: 0.9,
2444415021: 0.9,
1597098466341277696: 1.3,
1545402793493942272: 1.4,
1244497957518323712: 0.1,
2755716485: 4.3,
240900108: 1.5,
1154440893597999104: 1.3,
1575951085331795968: 2,
1506688480570712069: 2.2,
1260878504406716422: 0.6,
720749291749449728: 1.3,
1353012985380278272: 0.7,
1123228423567155202: 0.5,
1616676518125764609: 0.9,
1591986624023433216: 1.6,
235404787: 1.1,
154630645: 0.6,
2863339097: 1.1,
1539238611966734338: 1.9,
1458301412597985280: 1.3,
2255959788: 0.8,
109040003: 0.1,
1247924907486429184: 1.3,
1647875451053019137: 2.7,
304575140: 0.4,
1582725323547971589: 1.9,
1358485693362827264: 2.4,
501166414: 0.3,
1033150567240163328: 0.7,
425580734: 2,
169490045: 0.5,
1045725407440490496: 0.5,
1430824283886993410: 0.3,
3308887546: 0.4,
1479529413117198344: 1.2,
702092322: .1,
1626040766526021633: 1.5,
996443598777397250: 0.7,
1301931715707809792: 0.3,
884094616559079424: 1.8,
1622559160268103680: 2.4,
1830191916: 1.3,
1643620975378935809: 1.9,
1327250342187307012: 1.6,
1507992116: 0.1,
1374934140064305152: 1.2,
292533492: 0.4,
202204923: 1.4,
1399096477281030146: 1.6,
2798819219: 1.1,
32197321: 0.3,
1019909140934791169: 0.4,
1268238499599265806: 2.5,
1369802772665794560: 1.3,
2223877519: 0.2,
376222359: 0.6,
44088814: 0.1,
1489204488468529157: 1.7,
138054922: 0.4,
1590433892313354240: 2.6,
185335509: 0.4,
1576313209040019457: 1.1,
1460790813609402371: 0.7,
1459817387839131648: 1.9,
1561439341779484677: 1.2,
1966178407: 0.2,
1520346473845563394: 1.7,
207392880: 0.2,
1325455963: 0.4,
1377485913664667648: 1.3,
22942762: 0.7,
1408982517198163970: 0.4,
87508251: 1.1,
1489396994372157449: 1.6,
1351993328166846464: 3,
3166625095: 0.4,
1356988846081212420: 1.4,
50916923: 0.4,
1505292716489551874: 1.6,
1221429249195827203: 0.1,
1634808559383748609: 1.8,
854028653017149440: 1.7,
1519509449353961472: 0.8,
1098275124447301632: 2.7,
1319916015258537984: 0.3,
1323509574230421505: 0.3,
896808889831890944: 0.9,
1248164569: 3.5,
209136989: 2.0,
1598832068078755841: 2.4,
1369171181643862019: 0.5,
935578675868774400: 0.1,
1332769765795000325: 0.5,
1651130678229217280: 1.9,
60840932: 0.5,
1205380300588630021: 1.4,
}

# find botometer score for retweet-based coordination (we use default cutoff of 3)
hashtag_bot = {
1366765957587296267: 4.4,
1298654740021161985: 2.8,
1649232292928167936: 1.5,
1532008972672241665: 4.1,
114713313: 0.5,
1616971760351563781: 3.8,
1528782342290030593: 4.2,
1560019664570302464: 1.2,
1300737074774126594: 4.2,
945946299014303744: 1.7,
1300365611986542592: 4.3,
3365781148: 4.1,
1520404127519748096: 4.4,
1533542993566879746: 4.2,
2778326308: 3.1,
1573010026570321921: 4.1,
1391989620284432384: 2.1,
1483104575104110594: 2,
1532409214609522692: 2.9,
1593630045779673096: 2.8,
1523015920800698368: 3.8,
62802393: 0.8,
1132513724751785984: 0.2,
1139188359190892544: 0.9,
3853419857: 0.9,
1576517505895391232: 4.7,
4242778458: 4.3,
4776400513: 0.5,
1589464768124887040: 2.4,
1583736763352129536: 4.4,
1463479752711307266: 1,
1423962003757015042: 2,
1152291716998844421: 1.2,
1503705772294868992: 4.7,
1475762349042765827: 2.5,
1323557648025808898: 0.1,
1494306054771032065: 1.6,
1362031181890519041: 3.8,
1488359905476374528: 4,
1524751510797979648: 4.7,
1562113594338295809: 2.7,
1565046890764328964: 4.5,
105305259: 0.4,
281174511: 3.1,
1531147990164905984: 4.3,
1521379931137404929: 3.3,
1508450157465059336: 3.6,
1603407421887635456: 3.7,
1493108826782187525: 3.5,
1573009751054594048: 4.8,
1553255361209110528: 4,
1419319396460281860: 3.8,
1595876127385567235: 4.3,
1603542171134709761: 3.4,
1347785463566913541: 1.3,
1404835930955939841: 0.9,
1530999392211283971: 1.5,
2981685557: 3.4,
339208878: 0.5,
1576522023920336896: 4.9,
1702862604: 2,
1586084182337490944: 4.1,
1599172441586237440: 2,
2362613196: 0.1,
1522508809138225152: 2.7,
1561971708898131970: 4.5,
1700775313: 2.5,
833657097422147586:2.3,
1538492876417642496: 1.4,
1294299740583022592: 1.3,
1572050775899271168: 1.4,
1414615306828406798: 1,
1508465452313366533: 3.8,
1382783208: 4.8,
1469701027121307653: 1.2,
1503348653343322114: 2,
878388146492329988: 4,
1275956011501793280: 1.4,
1336043704243658758: 4.5,
1292830119174774784: 4,
215307951: 0.4,
1380779210654969857: 2.8,
1193499259955531777: 0.5,
1519560832400883712: 1.9,
1580109647234715648: 2.1,
1131348013409026049: 2.8,
1147007926965432320: 1.6,
1583379425998929920: 2.1,
1606193631915810816: 3,
2742329235: 4,
1595165624396926976: 2.1,
1594576624158388224: 4.5,
2467761588: 4.9,
1428676583993352200: 1.5,
1226805391553490945: 0.6,
1596694670352224256: 3.6,
1502838025625092098: 2,
578665215: 0.7,
1479095980351897602: 0.7,
1482493581088722945: 1.6,
}



# find botometer score for retweet-based coordination (we use default cutoff of 0.99)
retweet_bot = {
1482492223: 1.4,
981659486757838849: 0.9,
24823512: 0.7,
1508503638758592515: 2.9,
1023940182184288257: 1.2,
2409414559: 0.6,
1206515958791262208: 1.7,
413261107: 0.8,
2780346221: 1.3,
906108505513484289: 1.7,
879847801215221761: 1.7,
1531281431053230080: 2.8,
1199431126030401538: 2.0,
553952886: 1.6,
875430831334064128: 1.0,
3092438314: 1.4,
1545523991217147907: 3.7,
591179346: 1.4,
1487430991501303809: 3.6,
300634662: 2.3,
876093460389875712: 2.7,
153177381: 1.7,
894041585947615233: 1.6,
345981451: 1.8,
220844663: 0.8,
1318738336849272832: 1.4,
727614725882662912: 1.3,
62314888: 1.4,
122173354: 0.9,
54555176: 2.3,
748362288: 2.5,
1356012554: 2.1,
1440807361812205574: 1.8,
1388673710853279744: 1.3,
1257808704608460800: 2.1,
1311164323088101377: 0.6,
1486373982555492353: 0.9,
349084023: 1.9,
1012475880248143873: 2.8,
37725640: 0.9,
1369981060344340481: 1.9,
345457632: 0.9,
23510139: 1.6,
1284912511502950400: 1.6,
1399929747220488192: 0.8,
1320340688987410432: 3.4,
737483868714848261: 2.7,
26353005: 1.3,
1028312395326926848: 1.2,
1277613394867019776: 1.9,
3296160681: 1.3,
1475460328859312129: 1.4,
3199503629: 1.2,
964749355: 2.0,
31200476: 1.4,
1202288083581964288: 1.1,
2260402009: 2.0,
15726227: 0.6,
1342680692107268098: 2.2,
794156346371436544: 1.2,
244650352: 0.9,
2615044988: 1.3,
1905029168: 1.4,
1334320674106695683: 1.5,
1186667134107209728: 1.2,
22637747: 1.3,
1248638632669388801: 1.1,
535833722: 1.2,
546746195: 1.2,
2826494791: 0.9,
1203585853454503937: 2.1,
33913156: 0.6,
1510578974023249921: 2.4,
2333179337: 1.2,
703728938678423552: 2.3,
1586006376920268801: 2.0,
1187413831666065409: 1.3,
244973299: 2.0,
981670711: 1.0,
229201531: 1.1,
43345762: 1.0,
1347083001411166208: 1.8,
1414522131937906694: 2.0,
2917793049: 0.9,
750959971: 0.3,
283041807: 0.4,
1428212886036307972: 0.7,
1412339290311467011: 0.9,
1655198150: 1.0,
1491111468401831937: 2.0,
1066117214431035392: 1.4,
824216698551209984: 2.3,
898979912: 0.7,
1311657465570705408: 1.2,
1416710098513240066: 1.9,
176437698: 0.7,
2378611644: 1.7,
1205671738870136833: 2.3,
1184671074044133376: 1.1,
1317065769356824576: 1.6,
}

# find botometer score for retweet-based coordination (we use default cutoff of 0.99)
time_bot = {
1310438128637759488: 1.4,
992247472192868352: 1.1,
1019526090627350529: 2.0,
455248161: 1.5,
263164345: 1.3,
786118656: 2.3,
1379144882095280128: 1.3,
1539564943: 2.6,
1544734073964433408: 2.8,
1483921187189239810: 1.2,
2420712286: 1.7,
1317152831690969088: 3.1,
1316158478642294785: 3.4,
980418818: 4.3,
2265397625: 0.9,
633733572: 2.9,
1490807539923136520: 2.9,
2857046853: 1.6,
1067706126: 1.0,
1345319606802640896: 1.6,
901108721924878336: 2.8,
1438954847588528133: 1.6,
1233873815702134784: 1.2,
523102206: 3.0,
359752506: 0.5,
1420221394701328388: 2.4,
1506989240982425603: 2.4,
1264690847523770369: 0.9,
1578211535251554304: 2.8,
1482119816299397124: 1.4,
1430495136904060929: 2.0,
22086316: 0.9,
211830847: 0.9,
984617447121653767: 1.7,
753403115907473408: 1.9,
1439369756612734976: 1.7,
795742382411157504: 2.9,
253257813: 0.9,
228380252: 0.3,
1100425713008676866: 2.5,
1455325464646533124: 2.5,
243783661: 0.4,
406098567: 1.3,
791790123478241280: 1.3,
92441303: 0.7,
1407536840647794694: 2.2,
1260632966: 1.0,
1199713522785103872: 1.2,
37725640: 0.9,
15519565: 2.2,
1356998882534842371: 1.9,
35729673: 0.8,
1243348640585801728: 1.7,
1879161836: 0.7,
1249263211049861126: 1.2,
745774264255545344: 0.7,
1283351211735949313: 0.9,
242323327: 0.4,
618790274: 0.3,
50478299: 1.6,
332872124: 0.5,
1266104793820532738: 1.3,
1566305489918607361: 3.2,
1244602968873119744: 0.5,
974095389073682432: 1.4,
789122773499060224: 1.9,
1270133776748855304: 2.7,
228590514: 0.8,
14291408: 0.7,
160245128: 0.4,
2876378805: 1.4,
284737750: 0.9,
498645438: 3.4,
880624412: 2.0,
477855073: 0.8,
76560310: 1.6,
41438290: 2.4,
71026122: 0.4,
3269777071: 1.4,
848911132496723969: 3.1,
1493295859722764290: 1.7,
2636340443: 1.2,
3632161: 0.9,
7212562: 0.4,
1631457230921891840: 1.7,
1495662607679864835: 1.9,
195610523: 3.5,
1028951881882714114: 1.0,
1357444803911446534: 2.2,
1454861618572140548: 1.3,
1216524615310368769: 1.1,
40601473: 0.9,
860974889007685633: 2.1,
1667014841331228672: np.nan,
1564254826422308864: 1.4,
817188057380057088: 3.6,
147607161: 0.9,
788273299927150592: 0.9,
1297320697962921987: 1.4,
1239966468567003136: 1.3,
}

#https://botometer.osome.iu.edu/

random_before={
740194723185446912: 0.9,
939649716308103168: 0.3,
1519006366798684161: 0.7,
1530216009696952320: 1.2,
3319387244: 2,
1349602662275293184: 0.9,
1405098646056566785: 0.3,
1295732589064359937: 0.7,
863009146957967360: 4.5,
3292285374: 0.4,
1051565726400688128: 1.2,
1082038116335140866: 0.8,
29518506: 0,
3319538258: 2,
213638945: 0.9,
1477975707678560256: 0.7,
25362782: 0.4,
1320663862744502272: 0.4,
18912710: 1.3,
1538264090: 0.3,
984865785310072839: 1.1,
1087664364486905858: 1.4,
1109702122398990336: 0.6,
1145101815484166144: 0.1,
1254623707726860289: 2,
3266430876: 0.1,
2793302781: 0.1,
316208794: 0.1,
1194200010101051394: 1.4,
1563551511636230145: 2,
601407302: 0.4,
1438584319472066562: 1.6,
1272793454: 0.6,
2847113829: 0.7,
51302350: 0.4,
50369193: 0.1,
1338913653140099074: 1.2,
867762124235436033:1.2,
436378772: 0.1,
61214134: 0.2,
1239970381965869057: 1.6,
1342056657141063680: 1.1,
759218015019859973: 1,
1437608067353530368: 1.3,
1489010688781524993: 1.5,
31087423: 0.6,
1289950112383475712: 0.4,
1215443672805920768: 0.7,
1560121198167568385: 1.1,
1437166329761386497: 1.1,
1267220630979506181: 0.9,
109036745: 0.1,
1279107365057572864: 1,
1563789237396586496: 2.8,
40659071: 0.8,
1029117376619536384:0.1,
817648951641968640: 0.4,
1668984350: 0.7,
1391917651195310090:0.4 ,
160677978: 0.9,
4173915014: 1.2,
2230675952: 0.3,
760538714590224384: 0.3,
1484912351635972098: 0.6,
1355459993332551686: 0.4,
1275530809382862848: 1.1,
26048354: 0.2,
1358844213622935557: 0.1,
15301179: 0.9,
22645752: 0.3,
1125461350434705410: 0.5,
168415837: 0.2,
912340413738029056: 2,
1463874072983392261: 2,
75974320: 1.5,
1321429574366961665: 1,
3074603468: 0.1,
3012513962: 0.7,
1402611644: 1.4,
2669170388: 1.3,
1493029247321006087: 2,
2837956302: 0.9,
2992724509: 1.8,
918637819790348290: 0.9,
1469385401114763277: 0.8,
338563550: 0.1,
1390052362904117252: 1.4,
945484454969065472: 0.1,
1328033385999081472: 0.9,
1418116092774387712: 1.4,
702569346: 0.4,
799248606: 0.3,
1459523551522967560: 1.9,
1195530643410698241: 1.7,
859524223471042560: 1.4,
1530113009573675009: 4.3,
17261238: 0.1,
41720563: 0.7,
53305515: 0.4,
2970682325: 0.4,
}

# random accounts (regardless of activity) across all data
random_accounts={
2155531993: 0.2,
1313907053866283008: 0.8,
1517284000091492352: 1.2,
1606212284166164483: 1.7,
1433367891206942723: 0,
1519749527091257344: 0.9,
1168398081605459968: 0.1,
1234428895115579392: 1.7,
1221508591838007297: 1.5,
1499760474568548352: 1.1,
1445345278656196618: 0.4,
1464817014430568455: 1.1,
1416347559132647424: 2.3,
1421838490011058177: 0.9,
1499738355411689473: 0.9,
1545808297395109890: 1.6,
1265297416552660992: 0.9,
1506042273720840192: 1.6,
1142975156454940673: 0.3,
883109380534579200: 0.7,
2870585611: 0.4,
792551726754697216: 0.4,
732258043950469120: 0.2,
1372326959368171522: 0.3,
1447747462035173382: 1.5,
135601098: 0.4,
723479995: 0,
880751600599392256: 0.8,
2283510368: 0.2,
731562270703095809: 0.1,
2636855864: 0.9,
869062871677894657: 0.1,
1157177385495564288: 1.3,
272921344: 0.7,
1079471930405728256: 0.4,
1163941679247679488: 4.1,
20948837: 1.0,
852104772: 0.4,
108450755: 1.3,
1448838760570003458: 1.2,
1232083753121591296: 0.7,
1503691794994790400: 0.7,
613577350: 0.3,
1146040618747125761: 0.8,
96571283: 0.4,
1528458001455120384: 1.1,
1103685877497323520: 1.6,
44330293: 0.5,
1557989696625676290: 1.3,
1598674752415080449: 1.8,
747051322667589632: 1.6,
854847174: 0.4,
1346496663154794502: 0.4,
29825604: 0.8,
1254914162: 0.6,
1387252714657271808: 3.5,
82546360: 0.5,
33022883: 0.6,
2690121732: 4.5,
1445458906268925961: 0.7,
3370048743: 0,
397439511: 0.6,
2260672405: 0,
1480470554276732930: 1.4,
1359018111626670080: 0.8,
1529471977513226243: 2.0,
387010985: 2.6,
741341573158440960: 0.3,
1496385889139843075: 2.0,
4894371664: 1.0,
843137859875946496: 1.7,
1391694265789517824: 2.0,
1477680719895973891: 1.4,
1508607095280750597: 0.7,
20211418: 0.9,
891716118720761856: 1.7,
864550608648380416: 0.4,
361496218: 0.1,
83009793: 3.1,
1038840273537773568: 0.8,
1435017486508838914: 2.7,
4685072792: 1.6,
2379460765: 0.2,
23712638: 0.6,
1455874996182601731: 1.7,
1309492481730383877: 0.2,
338394082: 0.7,
1145622665014730752: 0.1,
1499752353041457169: 2.1,
1232028249070428160: 0.4,
1272043156347944961: 1.3,
1465877455118422020: 0.8,
1012024710: 0.1,
632015832: 0.4,
182869392: 0.4,
1410013072601890825: 1.3,
166080774: 0.3,
4036227682: 0.6,
384870156: 0.3,
1292396225455460353: 0.6,
}

random_after={
33677356: 0.4,
853634438: 0.9,
1625996971533037568: 1.9,
1538165148925448195: 4.1,
1446169175857274888: 0.9,
1394754136462213120: 2,
12664712: 1.6,
35445292: 0.6,
1518640886275579904: 0.7,
954112941548326912: 0.2,
216410111: 0.4,
2983854807: 0.4,
1132507428: 1.7,
963430877337448454: 1.8,
613327130: 0.4,
1435609653329764363: 1.3,
1599489144908046336: 1.4,
1598476155975344131: 1.3,
1328832021943107585: 1.4,
1241529203939393536:0.4,
1074244220142129152: 1.2,
1303327777387470850: 0.1,
317129017: 0.3,
1647554298409287680: 2.7,
1546690166160498688:0.9,
286092338: 0.4,
1457355793142009863: 1.6,
2792760286: 0.9,
1586416081274605568: 1.2,
1371966961: 2.1,
1012255895487565829: 0.4,
1580588717399580672: 0.7,
1566418739972943873: 3.9,
1337458386661756929: 1.9,
1656422930178273283: 2.8,
1505585932916781057: 3.2,
1227272379644153863: 1.3,
3103654198: 1.3,
1545518025981452288: 1.8,
94206711: 1.2,
1650886508751011840: 2.8,
1393927298055823366: 0.9,
1435148205826609159: 2.2,
1439714983991029776: 1.3,
1158882804903624704: 0.4,
1328300797621198849: 0.2,
1499219805521465345: 0.7,
1411218065715470336: 2.8,
1079225740078563328: 1.7,
1442154846623186946: 1.4,
1714675920: 0.4,
1211378302490824705: 0.8,
1272827973280751616: 1.4,
186520006: 0.8,
1037373433011556354: 1.4,
2493168218: 0.5,
1296669131920625664: 1.4,
1579482722522767361: 1.8,
1455951648669454346: 0.9,
932223369562525696: 0.1,
1594373071057702912: 1.4,
1529928092118138882: 1.0,
1194587598590791681: 0.5,
1052392940755083265: 2.3,
1491107835266093061: 1.4,
855710131: 0.6,
1479180802336436227: 0.5,
794435069528117248: 0.8,
243406334: 0.6,
467832568: 0.5,
1408021676139315203: 1.8,
1226844377278009345: 0.9,
1582396457646161922: 2.5,
3425137709: 0.1,
1250849681405591552: 0.1,
232327294: 0.6,
1519295851914481666: 2.5,
1567032525062832128: 3.0,
42546569: 2.1,
958878925911818241: 0.4,
1281009042866012160: 1.9,
1593455850739335170: 1.4,
66128674: 0.4,
553224712: 1.9,
1539613268167118850: 0.7,
3311122235: 0.9,
1183414602207563776: 1.1,
754646491596226560: 0.1,
24213703: 0.4,
899214407007383552: 0.9,
910482612887502849: 0.8,
1129042777621311488: 0.8,
2938614574: 0.2,
1267633896268709890: 2.0,
532644675: 0.6,
1304080870853181441: 0.4,
925187845936918528: 0.9,
1528514976658366464: 1.5,
1074917640689336320: 0.3,
42001952: 0.3,
}

# random accounts that ever posted a cashtag
random_cashtag={
891170454614003713: 0.1,
1355640076038975488: 0.6,
1452660460013117449: 2.7,
1455079124742119425: 0.6,
1592958269563207680: 3.2,
1431837327341473792: 2.9,
1376550060234833920: 1.1,
1540833942026207235: 2.4,
1427329958955917312: 1.7,
419413533: 0.4,
1459352444664795139: 0.9,
1408885043607506956: 0.4,
748592786: 4.3,
1621336168892866562: 3.2,
84027591: 0.6,
1537418630769414144: 3.2,
1297282466714202113: 0.5,
1412972072049991681: 0.9,
1654325025732988928: 2.9,
1519476024748216324: 0.9,
1444334729738739716: 1.0,
1633294996794224640: 1.6,
2944031849: 1.3,
1518954876688076800: 1.8,
1499072181745160205: 1.4,
1334689100658593794: 1.4,
1451223986939985923: 2.6,
1414134450842132480: 1.3,
1244192172506636289: 0.9,
1494046580118331395: 2.3,
1380615936332414976: 0.4,
1534908868366438400: 2.5,
1528486610479759360: 1.1,
1503990809670328324: 2.7,
1575030155155714048: 2.9,
2976428467: 0.6,
1044993905350115328: 1.4,
1160101394956849152: 0.1,
921077772071686144: 0.4,
1595447017677131777: 2.3,
785875241792712704: 0.3,
1336910980673118208: 0.9,
594318745: 0.9,
938205761221595136: 0.9,
1590931521874952192: 2.3,
3436186541: 0.8,
1498845369169416195: 3.1,
1427364417315000324: 1.9,
2752985632: 0.5,
1490616314356199428: 3.3,
1380692092272320515: 1.3,
1525715406702247936: 1.1,
1425734205334454274: 2.5,
3242253904: 2.8,
1338708212237864965: 0.1,
1556825531852529664: 1.5,
1128466451764056064: 0.9,
1539720367807275008: 1.7,
1371076800034476038: 1.9,
876220159366770689: 2.5,
1494404986008227842: 1.4,
1401833614438854662: 2.2,
1002844338743750656: 1.6,
3352952831: 0.7,
1418135345699328000: 2.7,
1525416190885101573: 0.9,
1593764361981550592: 2.5,
1341489136541306887: 0.6,
187969571: 0.1,
1603767670066671616: 1.7,
1553328384566087682: 2.2,
1364391078070206467: 0.9,
283965556: 1.7,
1411253069279498241: 2.7,
1647822549244432384: 1.5,
1491449318032785409: 1.7,
707671081755213826: 0.1,
1379297793127288832: 2.8,
1468361324195422208: 3.0,
1354157941193105410: 2.8,
428951254: 0.3,
1548631326424588289: 2.6,
1465832884992978946: 1.7,
1072746110374957057: 2.0,
1521370774434902016: 1.6,
1398154511126962178: 0.6,
1198608610911240192: 1.1,
1490920916532051969: 1.3,
1258004063075086337: 0.5,
1506549627172106248: 0.9,
969716112752553985: 0.1,
1602277502: 0.7,
886188503662514177: 0.3,
1466126205862752261: 1.3,
1490909175748050954: 2.1,
1369999863501361157: 3.2,
1484024256468234243: 2.3,
1554363309545050112: 1.6,
103064422: 0.4,
60269289: 1.4,
}



In [ ]:
def lighten_color(color, amount=0.5):
    """
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2])
def barplot_annotate_brackets(num1, num2, data, center, height, ax,yerr=None, dh=.05, barh=.05, fs=None, maxasterix=None):
    """ 
    Annotate barplot with p-values.

    :param num1: number of left bar to put bracket over
    :param num2: number of right bar to put bracket over
    :param data: string to write or number for generating asterixes
    :param center: centers of all bars (like plt.bar() input)
    :param height: heights of all bars (like plt.bar() input)
    :param yerr: yerrs of all bars (like plt.bar() input)
    :param dh: height offset over bar / bar + yerr in axes coordinates (0 to 1)
    :param barh: bar height in axes coordinates (0 to 1)
    :param fs: font size
    :param maxasterix: maximum number of asterixes to write (for very small p-values)
    """

    if type(data) is str:
        text = data
    else:
        # * is p < 0.05
        # ** is p < 0.005
        # *** is p < 0.0005
        # etc.
        text = ''
        p = .05

        while data < p:
            text += '*'
            p /= 10.

            if maxasterix and len(text) == maxasterix:
                break

        if len(text) == 0:
            text = 'n. s.'

    lx, ly = center[num1], height[num1]
    rx, ry = center[num2], height[num2]

    if yerr:
        ly += yerr[num1]
        ry += yerr[num2]

    ax_y0, ax_y1 = plt.gca().get_ylim()
    dh *= (ax_y1 - ax_y0)
    barh *= (ax_y1 - ax_y0)

    y = max(ly, ry) + dh

    barx = [lx, lx, rx, rx]
    bary = [y, y+barh, y+barh, y]
    mid = ((lx+rx)/2, y+barh)

    ax.plot(barx, bary, c='black')

    kwargs = dict(ha='center', va='bottom')
    if fs is not None:
        kwargs['fontsize'] = fs

    ax.text(*mid, text, **kwargs)
bot_sample = {
    'random':np.array([random_accounts[k] for k in random_accounts.keys() if not np.isnan(random_accounts[k])]),
    'hashtag':np.array([hashtag_bot[k] for k in hashtag_bot.keys() if not np.isnan(hashtag_bot[k])]),
    'retweet':np.array([retweet_bot[k] for k in retweet_bot.keys() if not np.isnan(retweet_bot[k])]),
    'time':np.array([time_bot[k] for k in time_bot.keys() if not np.isnan(time_bot[k])]),
}
str_ps =[]
for key in ['hashtag','retweet','time']:
    r,p = mannwhitneyu(bot_sample['random'],bot_sample[key])

    str_p = 'p='+str(round(p,3))
    if p < 0.001:
        str_p = 'p<0.001'
    str_ps.append(str_p)
means = np.array([np.mean(bot_sample[key]) for key in bot_sample.keys()])
errs =np.array([np.std(bot_sample[key])/np.sqrt(len(bot_sample[key])) for key in bot_sample.keys()])
labels = ['Random','Hashtags','Co-repost','Activity']
colors=['w','lightgray','cornflowerblue','greenyellow']
one_hot_means = [[0]*(k)+[means[k]]+[0]*(3-k) for k in range(4)]
matplotlib.rcParams['font.size']=14
matplotlib.rcParams['font.family']='Arial'
fig,ax = plt.subplots(1,2,figsize=(8,4))
for k in range(4):
    ax[0].bar(labels,one_hot_means[k],color='w',alpha=1,hatch=["","//","//","//"][k],edgecolor = ["black",None,None,None][k],linewidth=3)
for k in range(4):
    ax[0].bar(labels,one_hot_means[k],color=colors[k],alpha=1,hatch=["","//","//","//"][k],linewidth=3)
ax[0].errorbar(labels,means,yerr=errs,color='k',linestyle='')
ax[0].set_ylabel('Botomoter X Score')
for k in range(3):
    barplot_annotate_brackets(0, k+1, str_ps[k], range(4),np.array([2,2,2.6,3.1])+0.8,ax=ax[0])
ax[0].set_ylim([0,4.5])
ax[0].set_xticks([0,1, 2, 3])
ax[0].set_xticklabels(labels, rotation=30, ha='center')



print('Cashtag')
print(np.mean([random_cashtag[k] for k in random_cashtag.keys() if not np.isnan(random_cashtag[k])]))
print('Random',np.mean(bot_sample['random']))
print(mannwhitneyu(bot_sample['random'],[random_cashtag[k] for k in random_cashtag.keys() if not np.isnan(random_cashtag[k])]))


bot_sample = {
    'random_before':np.array([random_before[k] for k in random_before.keys() if not np.isnan(random_before[k])]),
    'random_after':np.array([random_after[k] for k in random_after.keys() if not np.isnan(random_after[k])]),
    'random_weighted_before':np.array([random_weighted_before[k] for k in random_weighted_before.keys() if not np.isnan(random_weighted_before[k])]),
    'random_weighted_after':np.array([random_weighted_after[k] for k in random_weighted_after.keys() if not np.isnan(random_weighted_after[k])])
}


str_ps =[]
for key in ['random_after']:
    r,p = mannwhitneyu(bot_sample['random_before'],bot_sample[key])

    str_p = 'p='+str(round(p,3))
    if p > 0.01:
        str_p = 'p='+str(round(p,2))
    if p > 0.1:
        str_p = 'p='+str(round(p,2))
    if p < 0.001:
        str_p = 'p<0.001'
    str_ps.append(str_p)
for key in ['random_weighted_after']:
    r,p = mannwhitneyu(bot_sample['random_weighted_before'],bot_sample[key])

    str_p = 'p='+str(round(p,3))
    if p > 0.01:
        str_p = 'p='+str(round(p,2))
    if p > 0.1:
        str_p = 'p='+str(round(p,2))
    if p < 0.001:
        str_p = 'p<0.001'
    str_ps.append(str_p)
means = np.array([np.mean(bot_sample[key]) for key in bot_sample.keys()])
errs =np.array([np.std(bot_sample[key])/np.sqrt(len(bot_sample[key])) for key in bot_sample.keys()])
labels = ['Random User\nBefore Purchase','Random User\nAfter Purchase','Random Post\nBefore Purchase','Random Post\nAfter Purchase']
labels = ['Pre-\nPurchase','Post-\nPurchase','Pre-\nPurchase','Post-\nPurchase']

colors=['#8DDDD0','#CA472F','#8DDDD0','#CA472F']
#8DDDD0, and the red one is #CA472F. I set alpha=0.9 
one_hot_means = [[0]*(k)+[means[k]]+[0]*(len(bot_sample.keys())-1-k) for k in range(len(bot_sample.keys()))]
print(one_hot_means)

for k in range(len(bot_sample.keys())):
    ax[1].bar([xx+(int(xx/2)*0.2) for xx in range(4)],one_hot_means[k],color='w',alpha=1,hatch=["","","o","o"][k],width=1)

for k in range(len(bot_sample.keys())):
    ax[1].bar([xx+(int(xx/2)*0.2) for xx in range(4)],one_hot_means[k],color=colors[k],alpha=0.9,hatch=["","","o","o"][k],width=1,label=['Pre-purchase','Post-purchase',None,None][k])
ax[1].errorbar([xx+(int(xx/2)*0.2) for xx in range(4)],means,yerr=errs,color='k',linestyle='')
ax[1].set_xticklabels(['']+labels, rotation=65)
ax[1].set_ylabel('Botomoter X Score')
ax[1].legend()
ax[1].set_ylim([0,2.1])
plt.xticks(rotation=30,size=12)
plt.tight_layout()
plt.savefig('RandomBotComparison.pdf',transparent=True)
plt.show()

bot_sample = {
    'random_before':np.array([random_before[k] for k in random_before.keys()]),
    'random_after':np.array([random_after[k] for k in random_after.keys()]),
    'random_weighted_before':np.array([random_weighted_before[k] for k in random_weighted_before.keys()]),
    'random_weighted_after':np.array([random_weighted_after[k] for k in random_weighted_after.keys()])
}
bot_sample = pd.DataFrame(bot_sample)
bot_sample2 = {'Botomoter X Score':[],'Purchase':[],'Sampling':[]}
for ii,c in enumerate(bot_sample.columns):
    bot_sample2['Botomoter X Score'] += bot_sample[c].values.tolist()
    b = ii%2
    bot_sample2['Purchase'] += [b]*100
    s = int(ii/2)
    bot_sample2['Sampling'] += [s]*100
bot_sample2 = pd.DataFrame(bot_sample2)
fig,ax = plt.subplots(1,1,figsize=(5,5))

sns.violinplot(y='Botomoter X Score',x='Purchase', data=bot_sample2, color="0.8")
sns.stripplot(y='Botomoter X Score',x='Purchase', data=bot_sample2, jitter=True,hue="Sampling")
plt.ylabel('Botomoter X Score')
plt.ylim([0,5])
plt.xticks(rotation=30)
plt.show()